In [146]:
############################################ 라이브러리 로드 #########################################
import requests
import time
import csv
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
#######################################################################################################


#별점을 정수화
def change_stars(stars):
    
    if stars == '1/5':
            rating = 1
    elif stars == '2/5':
        rating = 2
    elif stars == '3/5':
        rating = 3
    elif stars == '4/5':
        rating = 4
    else:
        rating = 5
            
    return rating


def app_review_crawling(url):
    
    #무한스크롤
    driver = webdriver.Chrome('/chromedriver')
    driver.get(url)
    driver.implicitly_wait(5)
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight") # 스크롤 높이 가져오기
    count = 0

    while True:
                                                            
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 아래까지 스크롤하기 
        time.sleep(SCROLL_PAUSE_TIME) # 페이지 로드 기다리기                                                
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        #50번 스크롤함
        if count == 50:                                                
            break

        last_height = new_height
        count += 1
    
    
    #데이터 파싱
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    div_tags = soup.select('div.l-row > div.ember-view')
    print(len(div_tags)) # 리뷰 개수
    temp_dict = {} 
    
    
    #딕셔너리에 값 넣기
    for i in range(len(div_tags)):
        
        header = div_tags[i].find(class_ = 'we-customer-review__header') # 하나의 리뷰박스
        date = header.find('time',class_ = 'we-customer-review__date').text
        title = div_tags[i].find('h3', class_='we-customer-review__title').get_text(strip = True)
        body_tags = div_tags[i].find('blockquote', class_='we-customer-review__body').find_all('p')
        body = ''
        for body_tag in body_tags:
            body += ' ' + body_tag.text
        body = body.lstrip()
        stars = div_tags[i].find('figure')['aria-label']
        rating = change_stars(stars)
        temp_dict[i] = {'작성날짜':date, '제목':title, '내용':body, '평점':rating}
    
    
    #데이터프레임 생성
    review_df = pd.DataFrame(temp_dict).transpose()
    
    return review_df

In [141]:
지그재그 = 'https://apps.apple.com/kr/app/%EC%A7%80%EA%B7%B8%EC%9E%AC%EA%B7%B8-no-1-%EC%97%AC%EC%84%B1%EC%87%BC%ED%95%91%EB%AA%B0-%EB%AA%A8%EC%9D%8C%EC%95%B1/id976131101?app=itunes&ign-mpt=uo%253D4#see-all/reviews'
에이블리 = 'https://apps.apple.com/kr/app/%EC%97%90%EC%9D%B4%EB%B8%94%EB%A6%AC-%EC%85%80%EB%9F%BD%EB%A7%88%EC%BC%93-%EB%AA%A8%EC%9D%8C%EC%95%B1/id1084960428#see-all/reviews'
브랜디 = 'https://apps.apple.com/kr/app/%EB%B8%8C%EB%9E%9C%EB%94%94-%EC%A0%84%EC%83%81%ED%92%88-%EB%AC%B4%EB%B0%B0/id1005442353#see-all/reviews'

In [142]:
raw_zigzag_df = app_review_crawling(지그재그)

940


In [143]:
raw_ably_df = app_review_crawling(에이블리)

820


In [144]:
raw_brandy_df = app_review_crawling(브랜디)

770


In [145]:
raw_zigzag_df.to_csv("raw_zigzag_data.csv")
raw_ably_df.to_csv('raw_ably_data.csv')
raw_brandy_df.to_csv('raw_brandy_data.csv')